In [4]:
import csv
import pandas as pd
def save_data(lips, Y, data_type):
    with open(f'{data_type}_lips.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ['label']
        for i in range(lips[0].shape[0]): #frames
            for j in range(lips[0].shape[1]): #height
                for k in range(lips[0].shape[2]): #width
                    header.append(f'pixel_{i}_{j}_{k}')
        writer.writerow(header)
        for i in range(len(lips)):
            row = [Y[i].argmax()]
            current_lips = lips[i].flatten()
            for j in range(len(current_lips)):
                row.append(current_lips[j])
            writer.writerow(row)
            
            
def load_data():
    X_train = pd.read_csv('train_lips.csv')
    Y_train = X_train['label']
    del X_train['label']
    X_valid = pd.read_csv('valid_lips.csv')
    Y_valid = X_valid['label']
    del X_valid['label']
    
    X_test = pd.read_csv('test_lips.csv')
    Y_test = X_test['label']
    del X_test['label']
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test

In [5]:
# loading the data from the csv files
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_data()
import numpy as np
import tensorflow
x_train = np.array(X_train, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_valid = np.array(X_valid, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_test = np.array(X_test, dtype='float32').reshape(-1, 11, 60, 100, 1)
nb_classes = 3



TypeError: 'tuple' object is not callable

In [7]:
y_train = tensorflow.keras.utils.to_categorical(Y_train, nb_classes)
y_valid = tensorflow.keras.utils.to_categorical(Y_valid, nb_classes)
y_test = tensorflow.keras.utils.to_categorical(Y_test, nb_classes)


In [22]:
x_train /= 255
x_test /= 255
x_valid /=255

In [23]:
import tensorflow as tf
model = tf.keras.models.Sequential()

In [24]:
model.add(tf.keras.layers.Conv3D(64, kernel_size=(4, 5, 5), activation='relu', input_shape=(11, 60, 100, 1)))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 3, 4)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_2 (Conv3D)               │ (None, 9, 56, 96, 64)  │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 4, 18, 24, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 18, 24, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 110592)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    14,155,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,169,475 (54.05 MB)

 Trainable params: 14,169,347 (54.05 MB)

 Non-trainable params: 128 (512.00 B)

In [28]:
es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=5)

In [29]:
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_valid, y_valid,), callbacks=[es])

Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.5960 - loss: 0.8618 - val_accuracy: 0.4098 - val_loss: 1.0723
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6500 - loss: 0.8057 - val_accuracy: 0.4098 - val_loss: 1.0685
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6720 - loss: 0.7482 - val_accuracy: 0.5082 - val_loss: 1.0102
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.7209 - loss: 0.6546 - val_accuracy: 0.4672 - val_loss: 1.0182
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.6838 - loss: 0.7554 - val_accuracy: 0.5246 - val_loss: 0.9840
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.7689 - loss: 0.5705 - val_accuracy: 0.6885 - val_loss: 0.9170
Epoch 6: early stopping


In [30]:
from sklearn.metrics import classification_report

preds = model.predict(x_test)

print(classification_report(np.argmax(y_test,1),np.argmax(preds,1)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
              precision    recall  f1-score   support

           0       0.58      1.00      0.74        50
           1       1.00      0.04      0.08        23
           2       1.00      0.72      0.84        50

    accuracy                           0.71       123
   macro avg       0.86      0.59      0.55       123
weighted avg       0.83      0.71      0.65       123


In [32]:
model.save('model.simple.h5')  

In [ ]:
model= tf.keras.models.load_model('model.simple')